In [1]:
import os
import cv2
import imutils
import mahotas as mt
import numpy as np
from sklearn import metrics
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report,confusion_matrix

In [17]:
#making the train dataset along with labels

gt_folder_path = r'C:\Users\Legion\Desktop\Internship_IIT\Training & esting Dataset\Datasets\Training01\Ground Truth'

orig_folder_path = r'C:\Users\Legion\Desktop\Internship_IIT\Training & esting Dataset\Datasets\Training01\Original Image'

segments = ['Bicycle', 'Car', 'Human']


#lists of data
train_label = []
hog_train_data = []
humoments_train_data = []


#encoding the label names into numbers
#human is 0
#car is 1
#bicycle is 2

encoded ={'human':0, 'car':1, 'bicycle':2}


#describing the HOG descriptor
hog = cv2.HOGDescriptor()

#initialising the brisk descriptor
brisk = cv2.BRISK_create()


for segment in segments:
    
    image_folder = os.path.join(orig_folder_path,segment)
    gt_image_folder = os.path.join(gt_folder_path,segment)
    
    
    list_orig = os.listdir(image_folder)
    list_gt = os.listdir(gt_image_folder)
    
    for orig_name,gt_name in zip(list_orig,list_gt):
        
        orig_image = cv2.imread(os.path.join(image_folder,orig_name),1)
        gt_image = cv2.imread(os.path.join(gt_image_folder,gt_name),0)
        
        if segment == 'Bicycle':
            gt_image = np.invert(gt_image)
            train_label.append(2)
        
        if segment == 'Human':
            train_label.append(0)
            
        if segment == 'Car':
            train_label.append(1)
        grey_scaled_image = cv2.cvtColor(orig_image,cv2.COLOR_BGR2GRAY)    
        edges = cv2.Canny(gt_image,230,250)
        
        contours,hierarchies = cv2.findContours(edges,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        max_cnt = max(contours,key=cv2.contourArea)
        
        max_rect = cv2.boundingRect(max_cnt)
        #cv2.drawContours(orig_image,contours,-1,(0,0,255),2)
        
        x,y,w,h = max_rect
        
        '''
        cv2.rectangle(orig_image,(x,y),(x+w,y+h),(0,0,255),1)
        cv2.imshow(orig_name,orig_image)
        cv2.imshow(gt_name,edges)
        k = cv2.waitKey(0) & 0xFF
        if k == ord('q'):
            cv2.destroyWindow(orig_name)
            cv2.destroyWindow(gt_name)
            break
        else:
            cv2.destroyWindow(orig_name)
            cv2.destroyWindow(gt_name)
        '''
        
        if segment == 'Bicycle':
            resized_patch = cv2.resize(edges,(64,128))
        else:
            image_patch = edges[y:y+h,x:x+w]
            resized_patch = cv2.resize(edges,(64,128))
                
        hog_feature = hog.compute(resized_patch)
        hog_train_data.append(hog_feature.reshape(3780))
        
        

In [18]:
# randomise the datasets: labels and data
randomise = np.arange(len(hog_train_data))
np.random.shuffle(randomise)

hog_train_data = np.asarray(hog_train_data)
hog_train_data = hog_train_data[randomise]

train_label = np.asarray(train_label)
train_label = train_label[randomise]

In [21]:
linsvm=SVC(kernel='linear',gamma='auto')
polysvm = SVC(kernel = 'poly',gamma='auto')
rbfsvm = SVC(kernel='rbf',gamma='auto')

linsvm.fit(hog_train_data,train_label)
polysvm.fit(hog_train_data, train_label)
rbfsvm.fit(hog_train_data,train_label)

orig_test_path = r'C:\Users\Legion\Desktop\Internship_IIT\Training & esting Dataset\Datasets\Testing01\Original'
gt_test_path = r'C:\Users\Legion\Desktop\Internship_IIT\Training & esting Dataset\Datasets\Testing01\GT'

#gt_test_path = r'C:\Users\Legion\Desktop\Internship_IIT\Training & esting Dataset\Datasets\Training01\Ground Truth\Car'
#orig_test_path = r'C:\Users\Legion\Desktop\Internship_IIT\Training & esting Dataset\Datasets\Training01\Original Image\Car'

list_orig_test = os.listdir(orig_test_path)
list_gt_test = os.listdir(gt_test_path)

#print(list_orig_test)
#print(list_gt_test)
bbox_encoding = {0:(0,0,255), 1:(0,255,0), 2:(255,0,0)}

hog=cv2.HOGDescriptor()

for orig_test_image_name,gt_test_image_name in zip(list_orig_test,list_gt_test):
    
    orig_test_image = cv2.imread(os.path.join(orig_test_path,orig_test_image_name),1)
    #print(os.path.join(orig_test_path,orig_test_image_name))
    
    gt_test_image = cv2.imread(os.path.join(gt_test_path,gt_test_image_name),0)
    #gt_test_image = np.invert(gt_test_image)
    #print(os.path.join(gt_test_path,gt_test_image_name))
    
    
    contours,hierarchies = cv2.findContours(gt_test_image , cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    rects = [cv2.boundingRect(cnt) for cnt in contours]
    
    patches = []
    
    for rect in rects:
        x,y,w,h = rect
        patch = orig_test_image[y:y+h,x:x+w]
        resized_patch = cv2.resize(patch,(64,128))
        
        hog_features=hog.compute(resized_patch).reshape(3780)
        
        y_pred = polysvm.predict([hog_features])
        
        #print(y_pred[0])
        
        cv2.rectangle(orig_test_image, (x,y),(x+w,y+h),bbox_encoding[y_pred[0]],1)
    cv2.imshow(orig_test_image_name,orig_test_image)
    k = cv2.waitKey(0) & 0xFF
    if k== ord('q'):
        cv2.destroyWindow(orig_test_image_name)
        break
    else :
        cv2.destroyWindow(orig_test_image_name)

In [1]:
##using the sklearn hog

In [1]:
import os
import cv2
import mahotas
import numpy as np
from skimage import exposure
from skimage import feature
import matplotlib.pyplot as plt
from sklearn.svm import SVC

In [2]:
gt_folder_path = r'C:\Users\Legion\Desktop\Internship_IIT\Training & esting Dataset\Datasets\Training01\Ground Truth'

orig_folder_path = r'C:\Users\Legion\Desktop\Internship_IIT\Training & esting Dataset\Datasets\Training01\Original Image'

segments = ['Bicycle', 'Car', 'Human']


#lists of data
train_label = []
skimage_hog_train_data = []


#encoding the label names into numbers
#human is 0
#car is 1
#bicycle is 2

encoded ={'human':0, 'car':1, 'bicycle':2}


#describing the HOG descriptor
hog = cv2.HOGDescriptor()

#initialising the brisk descriptor
brisk = cv2.BRISK_create()


for segment in segments:
    
    image_folder = os.path.join(orig_folder_path,segment)
    gt_image_folder = os.path.join(gt_folder_path,segment)
    
    
    list_orig = os.listdir(image_folder)
    list_gt = os.listdir(gt_image_folder)
    
    for orig_name,gt_name in zip(list_orig,list_gt):
        
        orig_image = cv2.imread(os.path.join(image_folder,orig_name),1)
        gt_image = cv2.imread(os.path.join(gt_image_folder,gt_name),0)
        
        if segment == 'Bicycle':
            gt_image = np.invert(gt_image)
            train_label.append(2)
        
        if segment == 'Human':
            train_label.append(0)
            
        if segment == 'Car':
            train_label.append(1)
        
        grey_scaled_image = cv2.cvtColor(orig_image,cv2.COLOR_BGR2GRAY)    
        
        edges = cv2.Canny(gt_image,230,250)
        
        contours,hierarchies = cv2.findContours(edges,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        max_cnt = max(contours,key=cv2.contourArea)
        
        max_rect = cv2.boundingRect(max_cnt)
        #cv2.drawContours(orig_image,contours,-1,(0,0,255),2)
        
        x,y,w,h = max_rect
        
        image_patch = grey_scaled_image[y:y+h,x:x+w]
        resized_patch = cv2.resize(image_patch,(64,128))
        
        H,h_image = feature.hog(resized_patch, orientations=9, pixels_per_cell=(10,10),
                       cells_per_block=(2,2), transform_sqrt=True, block_norm="L1",visualize = True)

        h_image = exposure.rescale_intensity(h_image, out_range=(0,255))
        h_image = h_image.astype('uint8')
        
        cv2.imshow('hog image',h_image)
        cv2.imshow('gt',gt_image)
        k=cv2.waitKey(0) & 0xFF
        if k==ord('q'):
            cv2.destroyWindow('hog_image')
            cv2.destroyWindow('gt')
            break
        else:
            cv2.destroyWindow('hog_image')
            cv2.destroyWindow('gt')
        
        skimage_hog_train_data.append(H)
        

In [11]:
type(skimage_hog_train_data[0])

numpy.ndarray

In [20]:
# randomise the datasets: labels and data
randomise = np.arange(len(skimage_hog_train_data))
np.random.shuffle(randomise)

skimage_hog_train_data = np.asarray(skimage_hog_train_data)
skimage_hog_train_data = skimage_hog_train_data[randomise]

train_label = np.asarray(train_label)
train_label = train_label[randomise]

In [26]:
linsvm=SVC(kernel='linear',gamma='auto')
polysvm = SVC(kernel = 'poly',gamma='auto')
rbfsvm = SVC(kernel='rbf',gamma='auto')

linsvm.fit(skimage_hog_train_data,train_label)
polysvm.fit(skimage_hog_train_data, train_label)
rbfsvm.fit(skimage_hog_train_data,train_label)

orig_test_path = r'C:\Users\Legion\Desktop\Internship_IIT\Training & esting Dataset\Datasets\Testing01\Original'
gt_test_path = r'C:\Users\Legion\Desktop\Internship_IIT\Training & esting Dataset\Datasets\Testing01\GT'

#gt_test_path = r'C:\Users\Legion\Desktop\Internship_IIT\Training & esting Dataset\Datasets\Training01\Ground Truth\Car'
#orig_test_path = r'C:\Users\Legion\Desktop\Internship_IIT\Training & esting Dataset\Datasets\Training01\Original Image\Car'

list_orig_test = os.listdir(orig_test_path)
list_gt_test = os.listdir(gt_test_path)

#print(list_orig_test)
#print(list_gt_test)
bbox_encoding = {0:(0,0,255), 1:(0,255,0), 2:(255,0,0)}

hog=cv2.HOGDescriptor()

for orig_test_image_name,gt_test_image_name in zip(list_orig_test,list_gt_test):
    
    orig_test_image = cv2.imread(os.path.join(orig_test_path,orig_test_image_name),1)
    #print(os.path.join(orig_test_path,orig_test_image_name))
    
    gt_test_image = cv2.imread(os.path.join(gt_test_path,gt_test_image_name),0)
    #gt_test_image = np.invert(gt_test_image)
    #print(os.path.join(gt_test_path,gt_test_image_name))
    
    
    
    contours,hierarchies = cv2.findContours(gt_test_image , cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    rects = [cv2.boundingRect(cnt) for cnt in contours]
    
        
    for rect in rects:
        x,y,w,h = rect
        patch = orig_test_image[y:y+h,x:x+w]
        resized_patch = cv2.resize(patch,(64,128))
        
        hog_features,h_image =feature.hog(resized_patch, orientations=9, pixels_per_cell=(10, 10),
                                 cells_per_block=(2, 2), transform_sqrt=True, block_norm="L1",visualize=True)        
        y_pred = rbfsvm.predict(hog_features.reshape(1,-1))
        
        print(y_pred[0])
        
        h_image = exposure.rescale_intensity(h_image,out_range=(0,255))
        h_image = h_image.astype('uint8')
        cv2.imshow('hog patch',h_image)
        cv2.imshow('patch' ,resized_patch)
        
        k = cv2.waitKey(0) & 0xFF
        if k== ord('q'):
            cv2.destroyWindow('patch')
            cv2.destroyWindow('hog_patch')
            break
        else :
            cv2.destroyWindow('patch')
            cv2.destroyWindow('hog_patch')
        
        cv2.rectangle(orig_test_image, (x,y),(x+w,y+h),bbox_encoding[y_pred[0]],1)
    cv2.imshow(orig_test_image_name,orig_test_image)
    k = cv2.waitKey(0) & 0xFF
    if k== ord('q'):
        cv2.destroyWindow(orig_test_image_name)
        break
    else :
        cv2.destroyWindow(orig_test_image_name)

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
